In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 
     |████████████████████████████████| 880 kB 68.9 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=20256e9fb0ebda5a2b66693323681528c8144c025b5ccc493877ebde3c6d27b9
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F


In [ ]:

!mkdir pytorch

In [ ]:
!cp /content/drive/MyDrive/pytorch_model.bin /content/pytorch

In [ ]:
!cp /content/drive/MyDrive/ftb5/config.json /content/pytorch

In [ ]:
from transformers import BertModel
model = BertModel.from_pretrained('/content/pytorch')

Some weights of the model checkpoint at /content/pytorch were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import BertTokenizer


In [ ]:
model_name = '/content/pytorch/hindi-vocab.txt'
tokenizer = BertTokenizer.from_pretrained(model_name, truncation=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1656: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [ ]:
import pandas as pd
import pickle

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/INGL_Hinglish/train.csv")
print(df.shape)
df.head()

(2766, 5)


,English,Hindi,Hinglish,Average rating,Disagreement
0,Program module is a file that contains instruc...,"माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...","module , ek program hoti hai , jismen ya to so...",7,6
1,And to Thamud We sent their brother Sali 'h. H...,और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...,aur hamne aume samood ke pas unke bhaee saleh ...,6,4
2,"and, when reminded, do not remember\n","और जब उन्हें याद दिलाया जाता है, तो वे याद नही...","aur jab unhen yad dilaya jata hai , to ve yad ...",10,0
3,you won the TED Prize 2011.\n,तुम्हें २०११ का टेड प्राइज़ मिल गया है.\n,tumhen २०११ ka ted prize mil gaya hai\n,9,1
4,He gone to Kerodemal College of Delhi Universi...,उन्होंने बाद अध्ययन करने के लिए ये दिल्ली विश्...,unhonne bad science karne ke lie ye delhi univ...,7,0


In [ ]:
df_actual_test = pd.read_csv("/content/drive/MyDrive/INGL_Hinglish/test - no labels.csv")

In [ ]:
import pickle
with open('/content/drive/MyDrive/INGL_Hinglish/train_human_generated.pkl', 'rb') as handle:
    b = pickle.load(handle)

In [ ]:
b

In [ ]:
import pickle
with open('/content/drive/MyDrive/INGL_Hinglish/valid_human_generated.pkl', 'rb') as handle:
    b_val = pickle.load(handle)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
sentences = list(df.Hinglish.values)


In [ ]:
total_sentences = []
total_embeddings = []
for sen in sentences:
  encoded_input = tokenizer(sen, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
  total_sentences.append(sen)
  total_embeddings.append(sentence_embeddings)
  

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
len(total_embeddings)


2766

In [ ]:
train = pd.DataFrame(zip(total_sentences,total_embeddings),columns = ['Sentence','Embeddings_Hinglish'])

In [ ]:
train.head()

,Sentence,Embeddings_Hinglish
0,"module , ek program hoti hai , jismen ya to so...","[[tensor(-0.0413), tensor(-0.0456), tensor(-0...."
1,aur hamne aume samood ke pas unke bhaee saleh ...,"[[tensor(0.0217), tensor(0.0325), tensor(-0.04..."
2,"aur jab unhen yad dilaya jata hai , to ve yad ...","[[tensor(0.0412), tensor(-0.0153), tensor(-0.0..."
3,tumhen २०११ ka ted prize mil gaya hai\n,"[[tensor(0.0210), tensor(-0.0213), tensor(-0.0..."
4,unhonne bad science karne ke lie ye delhi univ...,"[[tensor(0.0270), tensor(0.0337), tensor(-0.05..."


In [ ]:
train['Embeddings_Hinglish'] = train['Embeddings_Hinglish'].apply(lambda x: x[0].numpy())

In [ ]:
# train.Embeddings_Hinglish.values[0][0].numpy()

In [ ]:
df_actual_test.head()

,English,Hindi,Hinglish
0,Late Puranchandra Gupta never allowed any powe...,स्वर्गीय पूरनचन्द्र गुप्ता कभी भी अपने समाचारप...,late puranchandra gupta kabhi bhi apne press s...
1,From popular home stays where one can get a fi...,यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण...,yahan ke popular home aashray jahan aapko gram...
2,Compete with one another to achieve forgivenes...,तुम अपने परवरदिगार के (सबब) बख़्शिस की और बेहि...,tum apne paravardigar ke sabab bashis ki aur b...
3,Legislators must always remember that as repre...,विधायकों को सदैव याद रखना चाहिए कि जन प्रतिनिध...,legislators ko sadaiv yad people chahie ki pub...
4,The cap is designed to create an almost air ti...,कैप को इस तरह बनाया गया है कि यह गर्भाशय के मु...,cap ko is tarah banaya gaya hai ki yah garbhas...


In [ ]:
sentences = list(df_actual_test.Hinglish.values)

In [ ]:
total_sentences = []
total_embeddings = []
for sen in sentences:
  encoded_input = tokenizer(sen, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
  total_sentences.append(sen)
  total_embeddings.append(sentence_embeddings)
  

In [ ]:
test = pd.DataFrame(zip(total_sentences,total_embeddings),columns = ['Sentence','Embeddings_Hinglish'])

In [ ]:
test.head()

,Sentence,Embeddings_Hinglish
0,late puranchandra gupta kabhi bhi apne press s...,"[[tensor(0.0449), tensor(0.0392), tensor(-0.03..."
1,yahan ke popular home aashray jahan aapko gram...,"[[tensor(0.0267), tensor(0.0059), tensor(-0.02..."
2,tum apne paravardigar ke sabab bashis ki aur b...,"[[tensor(0.0011), tensor(0.0416), tensor(-0.03..."
3,legislators ko sadaiv yad people chahie ki pub...,"[[tensor(-0.0341), tensor(0.0123), tensor(-0.0..."
4,cap ko is tarah banaya gaya hai ki yah garbhas...,"[[tensor(-0.0255), tensor(0.0026), tensor(-0.0..."


In [ ]:
test['Embeddings_Hinglish'] = test['Embeddings_Hinglish'].apply(lambda x: x[0].numpy())

In [ ]:
train.to_csv("train_hinglish.csv",index = False)
test.to_csv("test_hinglish.csv",index = False)

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 1.2 MB 10.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=7effce840ec5ef59fe1898c8eb00a249fbefda75d7ad2122c396fbbc3e5021cb
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
df.head(2)

,English,Hindi,Hinglish,Average rating,Disagreement
0,Program module is a file that contains instruc...,"माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...","module , ek program hoti hai , jismen ya to so...",7,6
1,And to Thamud We sent their brother Sali 'h. H...,और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...,aur hamne aume samood ke pas unke bhaee saleh ...,6,4


In [ ]:
df_actual_test.head(2)

,English,Hindi,Hinglish
0,Late Puranchandra Gupta never allowed any powe...,स्वर्गीय पूरनचन्द्र गुप्ता कभी भी अपने समाचारप...,late puranchandra gupta kabhi bhi apne press s...
1,From popular home stays where one can get a fi...,यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण...,yahan ke popular home aashray jahan aapko gram...


In [ ]:
from sentence_transformers import SentenceTransformer
sentences = list(df.English.values)
model = SentenceTransformer('sentence-transformers/LaBSE',device = 'cuda')
embeddings = model.encode(sentences)


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

In [ ]:
train_English = pd.DataFrame(zip(sentences,embeddings),columns = ['Sentence','Embeddings_English'])

In [ ]:
train_English.head()

,Sentence,Embeddings_English
0,Program module is a file that contains instruc...,"[-0.015310467, -0.024479816, -0.029497672, -0...."
1,And to Thamud We sent their brother Sali 'h. H...,"[-0.0651833, -0.0078069284, -0.0398715, -0.003..."
2,"and, when reminded, do not remember\n","[-0.040339008, 0.008347899, 0.047642138, -0.02..."
3,you won the TED Prize 2011.\n,"[-0.04929146, -0.037394114, -0.008753206, -0.0..."
4,He gone to Kerodemal College of Delhi Universi...,"[-0.028583063, -0.046409346, -0.054248054, -0...."


In [ ]:
train_English

,Sentence,Embeddings_English
0,Program module is a file that contains instruc...,"[-0.015310467, -0.024479816, -0.029497672, -0...."
1,And to Thamud We sent their brother Sali 'h. H...,"[-0.0651833, -0.0078069284, -0.0398715, -0.003..."
2,"and, when reminded, do not remember\n","[-0.040339008, 0.008347899, 0.047642138, -0.02..."
3,you won the TED Prize 2011.\n,"[-0.04929146, -0.037394114, -0.008753206, -0.0..."
4,He gone to Kerodemal College of Delhi Universi...,"[-0.028583063, -0.046409346, -0.054248054, -0...."
...,...,...
2761,Polar ice caps may melt further and increase t...,"[-0.022657828, 0.033098616, -0.004224084, 0.00..."
2762,"It ' s what turns lead into gold , and makes t...","[-0.06607383, -0.038033247, -0.007904237, -0.0..."
2763,The President said the North Eastern Hill Univ...,"[0.0007948883, 0.027576888, -0.035335038, -0.0..."
2764,The violin bow might well have grown out of th...,"[0.069327325, 0.049504433, -0.033051636, 0.008..."


In [ ]:
train_English.to_csv("train_english.csv",index = False)


In [ ]:
from sentence_transformers import SentenceTransformer
sentences = list(df.Hindi.values)
model = SentenceTransformer('sentence-transformers/LaBSE',device = 'cuda')
embeddings = model.encode(sentences)

In [ ]:
train_Hindi = pd.DataFrame(zip(sentences,embeddings),columns = ['Sentence','Embeddings_Hindi'])

In [ ]:
train_Hindi.to_csv("train_Hindi.csv",index = False)

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = list(df_actual_test.English.values)
model = SentenceTransformer('sentence-transformers/LaBSE',device = 'cuda')
embeddings = model.encode(sentences)


In [ ]:
test_English = pd.DataFrame(zip(sentences,embeddings),columns = ['Sentence','Embeddings_English'])

In [ ]:
test_English.to_csv("test_English.csv",index = False)

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = list(df_actual_test.Hindi.values)
model = SentenceTransformer('sentence-transformers/LaBSE',device = 'cuda')
embeddings = model.encode(sentences)

In [ ]:
test_Hindi = pd.DataFrame(zip(sentences,embeddings),columns = ['Sentence','Embeddings_Hindi'])

In [ ]:
test_Hindi.to_csv("test_Hindi.csv",index = False)

In [ ]:
df.head(2)

,English,Hindi,Hinglish,Average rating,Disagreement
0,Program module is a file that contains instruc...,"माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...","module , ek program hoti hai , jismen ya to so...",7,6
1,And to Thamud We sent their brother Sali 'h. H...,और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...,aur hamne aume samood ke pas unke bhaee saleh ...,6,4


In [ ]:
df_actual_test.head(2)

,English,Hindi,Hinglish
0,Late Puranchandra Gupta never allowed any powe...,स्वर्गीय पूरनचन्द्र गुप्ता कभी भी अपने समाचारप...,late puranchandra gupta kabhi bhi apne press s...
1,From popular home stays where one can get a fi...,यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण...,yahan ke popular home aashray jahan aapko gram...


In [ ]:
df['Embeddings_Hinglish'] = train.Embeddings_Hinglish.values
df_actual_test['Embeddings_Hinglish'] = test.Embeddings_Hinglish.values
df['Embeddings_English'] = train_English.Embeddings_English.values
df_actual_test['Embeddings_English'] = test_English.Embeddings_English.values
df['Embeddings_Hindi'] = train_Hindi.Embeddings_Hindi.values
df_actual_test['Embeddings_Hindi'] = test_Hindi.Embeddings_Hindi.values

In [ ]:
df.head()

,English,Hindi,Hinglish,Average rating,Disagreement,Embeddings_Hinglish,Embeddings_English,Embeddings_Hindi
0,Program module is a file that contains instruc...,"माड्यूल, एक संचिका होती है, जिसमें या तो स्रोत...","module , ek program hoti hai , jismen ya to so...",7,6,"[-0.041341897, -0.04555844, -0.06375096, 0.059...","[-0.015310467, -0.024479816, -0.029497672, -0....","[-0.030017138, -0.05537624, -0.04613465, -0.05..."
1,And to Thamud We sent their brother Sali 'h. H...,और (हमने) क़ौमे समूद के पास उनके भाई सालेह को ...,aur hamne aume samood ke pas unke bhaee saleh ...,6,4,"[0.021724822, 0.0324799, -0.048921563, -0.0153...","[-0.0651833, -0.0078069284, -0.0398715, -0.003...","[-0.023371786, -0.03599291, -0.045930568, 0.01..."
2,"and, when reminded, do not remember\n","और जब उन्हें याद दिलाया जाता है, तो वे याद नही...","aur jab unhen yad dilaya jata hai , to ve yad ...",10,0,"[0.041205354, -0.015306432, -0.014218077, 0.03...","[-0.040339008, 0.008347899, 0.047642138, -0.02...","[-0.046261817, -0.016769359, 0.04940672, -0.01..."
3,you won the TED Prize 2011.\n,तुम्हें २०११ का टेड प्राइज़ मिल गया है.\n,tumhen २०११ ka ted prize mil gaya hai\n,9,1,"[0.020969704, -0.021344533, -0.02390608, -0.02...","[-0.04929146, -0.037394114, -0.008753206, -0.0...","[-0.043387998, 0.006866912, -0.01843704, -0.06..."
4,He gone to Kerodemal College of Delhi Universi...,उन्होंने बाद अध्ययन करने के लिए ये दिल्ली विश्...,unhonne bad science karne ke lie ye delhi univ...,7,0,"[0.027048927, 0.033708394, -0.059786793, -0.00...","[-0.028583063, -0.046409346, -0.054248054, -0....","[-0.033970833, -0.05304998, -0.04286438, -0.06..."


In [ ]:
df_actual_test.head()

,English,Hindi,Hinglish,Embeddings_Hinglish,Embeddings_English,Embeddings_Hindi
0,Late Puranchandra Gupta never allowed any powe...,स्वर्गीय पूरनचन्द्र गुप्ता कभी भी अपने समाचारप...,late puranchandra gupta kabhi bhi apne press s...,"[0.04493029, 0.039237022, -0.03795856, 0.03461...","[0.024422659, 0.022700377, -0.056133118, -0.00...","[0.0077234353, 0.0055396594, -0.05262459, 0.00..."
1,From popular home stays where one can get a fi...,यहां के लोकप्रिय घरेलू आश्रय जहां आपको ग्रामीण...,yahan ke popular home aashray jahan aapko gram...,"[0.026655046, 0.0059114243, -0.02049916, 0.022...","[0.051849727, 0.02399436, 0.043756347, -0.0181...","[0.049662646, -0.0035862897, -0.022955893, -0...."
2,Compete with one another to achieve forgivenes...,तुम अपने परवरदिगार के (सबब) बख़्शिस की और बेहि...,tum apne paravardigar ke sabab bashis ki aur b...,"[0.0011428738, 0.041595746, -0.032151606, -0.0...","[-0.06862237, -0.05251071, -0.02227133, 0.0107...","[0.014534808, -0.046254624, -0.047369882, 0.01..."
3,Legislators must always remember that as repre...,विधायकों को सदैव याद रखना चाहिए कि जन प्रतिनिध...,legislators ko sadaiv yad people chahie ki pub...,"[-0.03410484, 0.012294842, -0.034380816, 0.051...","[-0.05306453, -0.056643087, -0.03265491, -0.04...","[-0.054194298, -0.049915988, -0.042591676, -0...."
4,The cap is designed to create an almost air ti...,कैप को इस तरह बनाया गया है कि यह गर्भाशय के मु...,cap ko is tarah banaya gaya hai ki yah garbhas...,"[-0.025510808, 0.0026355984, -0.059162512, 0.0...","[-0.024522906, -0.053133912, 0.016547881, -0.0...","[-0.043405615, -0.041221216, -0.054696165, -0...."


In [ ]:
df.to_csv("train_main_INLG.csv",index = False)
df_actual_test.to_csv("test_main_INLG.csv",index = False)

In [ ]:
# !mkdir /content/drive/MyDrive/INLG_DATA

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
import numpy as np


In [ ]:
np.array(list(df.Embeddings_Hinglish.values[0])).shape

(768,)

In [ ]:
hng_em = []
for em in df.Embeddings_Hinglish.values:
  hng_em.append(em)


In [ ]:
np.array(hng_em).shape

(2766, 768)

In [ ]:
eng_em = []
for em in df.Embeddings_English.values:
  eng_em.append(em)


In [ ]:
hin_em = []
for em in df.Embeddings_Hindi.values:
  hin_em.append(em)


In [ ]:
training_embeddings = np.concatenate((np.array(hng_em),np.array(eng_em),np.array(hin_em)), axis=1)

In [ ]:
from catboost import CatBoostClassifier, Pool
model = CatBoostClassifier()
model.fit(training_embeddings, df['Average rating'].values)

Learning rate set to 0.083341
0:	learn: 2.1542322	total: 6.4s	remaining: 1h 46m 30s
1:	learn: 2.1160698	total: 10.3s	remaining: 1h 25m 47s
2:	learn: 2.0812476	total: 13.9s	remaining: 1h 16m 48s
3:	learn: 2.0516452	total: 17.3s	remaining: 1h 11m 43s
4:	learn: 2.0237789	total: 20.7s	remaining: 1h 8m 48s
5:	learn: 1.9990430	total: 24.1s	remaining: 1h 6m 39s
6:	learn: 1.9756496	total: 27.7s	remaining: 1h 5m 32s
7:	learn: 1.9550645	total: 32.2s	remaining: 1h 6m 38s
8:	learn: 1.9361776	total: 35.9s	remaining: 1h 5m 56s
9:	learn: 1.9176093	total: 39.5s	remaining: 1h 5m 10s
10:	learn: 1.9012670	total: 43.1s	remaining: 1h 4m 36s
11:	learn: 1.8858711	total: 46.5s	remaining: 1h 3m 49s
12:	learn: 1.8715437	total: 50s	remaining: 1h 3m 17s
13:	learn: 1.8566039	total: 53.6s	remaining: 1h 2m 53s
14:	learn: 1.8436547	total: 57.1s	remaining: 1h 2m 32s
15:	learn: 1.8319314	total: 1m	remaining: 1h 2m 14s
16:	learn: 1.8192403	total: 1m 4s	remaining: 1h 1m 58s
17:	learn: 1.8082111	total: 1m 7s	remaining: 1h

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier()
xgb_model.fit(training_embeddings, df['Average rating'].values)

XGBClassifier(objective='multi:softprob')

In [ ]:
hng_em_test = []
for em in df_actual_test.Embeddings_Hinglish.values:
  hng_em_test.append(em)


In [ ]:
eng_em_test = []
for em in df_actual_test.Embeddings_English.values:
  eng_em_test.append(em)


In [ ]:
hin_em_test = []
for em in df_actual_test.Embeddings_Hindi.values:
  hin_em_test.append(em)


In [ ]:
testing_embeddings = np.concatenate((np.array(hng_em_test),np.array(eng_em_test),np.array(hin_em_test)), axis=1)

In [ ]:
preds_1_cb = model.predict(testing_embeddings)

In [ ]:
preds_1_xgb = xgb_model.predict(testing_embeddings)

In [ ]:
from catboost import CatBoostClassifier, Pool
model = CatBoostClassifier()
model.fit(training_embeddings, df['Disagreement'].values)

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier()
xgb_model.fit(training_embeddings, df['Disagreement'].values)

XGBClassifier(objective='multi:softprob')

In [ ]:
preds_2_xgb = xgb_model.predict(testing_embeddings)

In [ ]:
preds_2_cb = model.predict(testing_embeddings)

In [ ]:
len(preds_2)

791

In [ ]:
len(preds_1)

791

In [ ]:
preds_1[:2]

array([7, 8])

In [ ]:
def maper(l):
  return l[0]

In [ ]:
preds_1_cb = list(map(maper,preds_1_cb))

In [ ]:
# file1 = open("myfile.txt", "a")  # append mode
# file1.write("Today \n")
# # file1.close()

In [ ]:
with open('submission_test.txt', 'w') as f:
    for item in preds_1:
        f.write("%s\n" % item)

In [ ]:
preds_2_cb = list(map(maper,preds_2_cb))

In [ ]:
with open('submission_test.txt', 'a') as f:
    for item in preds_2:
        f.write("%s\n" % item)

In [ ]:
# while True:
#   pass

In [ ]:
!unzip /content/submission_val.zip

Archive:  /content/submission_val.zip
  inflating: submission_val.txt      
   creating: __MACOSX/
  inflating: __MACOSX/._submission_val.txt  


Ensemble Not good


In [ ]:
df_1 = pd.DataFrame(zip(preds_1_xgb,preds_1_cb),columns = ['xgb','cb'])

In [ ]:
df_1.head()

,xgb,cb
0,3,3
1,1,1
2,1,1
3,2,0
4,2,2


In [ ]:
df_1['mean_rows'] = df_1.mean(axis = 1)

In [ ]:
df_1.head()

,xgb,cb,mean_rows
0,3,3,3.0
1,1,1,1.0
2,1,1,1.0
3,2,0,1.0
4,2,2,2.0


In [ ]:
df_1['mean_rows'] = df_1['mean_rows'].apply(lambda x:int(x))

In [ ]:
df_1.head()

,xgb,cb,mean_rows
0,3,3,3
1,1,1,1
2,1,1,1
3,2,0,1
4,2,2,2


In [ ]:
df_2 = pd.DataFrame(zip(preds_2_xgb,preds_2_cb),columns = ['xgb','cb'])

In [ ]:
df_2['mean_rows'] = df_2.mean(axis = 1)

In [ ]:
df_2['mean_rows'] = df_2['mean_rows'].apply(lambda x:int(x))

In [ ]:
with open('submission_test.txt', 'w') as f:
    for item in df_1.mean_rows.values:
        f.write("%s\n" % item)

In [ ]:
with open('submission_test.txt', 'a') as f:
    for item in df_2.mean_rows.values:
        f.write("%s\n" % item)